In [16]:

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
import skimage
from skimage.io import imread
from skimage.transform import resize
import pandas as pd
from sklearn.preprocessing import StandardScaler
import cv2
import glob

In [17]:
def load_image_files(container_path, dimension=(60, 60)):
    """
    Load image files with categories as subfolder names 
    which performs like scikit-learn sample dataset
    
    Parameters
    ----------
    container_path : string or unicode
        Path to the main folder holding one subfolder per category
    dimension : tuple
        size to which image are adjusted to
        
    Returns
    -------
    Bunch
    """
    scaler = StandardScaler()
    image_dir = Path(container_path)
    
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():#                       
            file_name ="C:/Users/utilisateur/OneDrive/Documents/Deeplearning/Altran/Treeth_Detection/"+str(file)
            img = cv2.imread(file_name,0) 
            # img = cv2.imread(file_name)           
            img_resized = cv2.resize(img, dimension)            
            img_resized = scaler.fit_transform(img_resized)            
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [18]:
image_dataset = load_image_files("images_svm/")

In [19]:
# df = pd.DataFrame(image_dataset.data)

In [20]:
# df.head()

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.2,random_state=109)

In [22]:
X = image_dataset.data
y = image_dataset.target

In [23]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier

# pca = PCA(n_components=2, svd_solver='randomized', whiten=True, random_state=42)
# pca.fit_transform(X_train, y_train)

####################################################################""
# params = {'C': [10], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'gamma': [0.0001]} 
# params = {'C': [10], 'kernel': ['linear'],'gamma': [0.0001]}
 
# model = svm.SVC()

#################################################""

params = {'alpha' : [.001, .01],
              'max_iter' : [10000, 30000],
              'tol' : [ .0001],
              'loss' : ['hinge', 'log', 'modified_huber'],
              'n_jobs' : [-1]}

model = SGDClassifier()

#################################################

# params = {'n_neighbors' : [3,5,11,19],
#           'weights' : ['uniform', 'distance'],
#           }

# model = KNeighborsClassifier()

###############################################################

# params = {}

# model = BaggingClassifier()

################################################################

# model = GridSearchCV(model, params, verbose=1, n_jobs=-1)

# model = GridSearchCV(model, param, verbose=1, n_jobs=-1)

# model = GridSearchCV(model, param_grid, verbose=1, n_jobs=-1)


# model = GridSearchCV(model, parameters, verbose=1, n_jobs=-1)



In [24]:
df_grid = pd.DataFrame(columns=['cv','score','best_params','grid'])
index = 0


grid = GridSearchCV(model, param_grid=params, verbose = 1, n_jobs = -1)
grid.fit(X,y)    
    

# for k in range(5,15) :
#     grid = GridSearchCV(model, param_grid=params, cv=k, verbose = 1, n_jobs = -1)
#     grid.fit(X_train,y_train)    
#     df_grid.loc[index] = [k,grid.score(X_test,y_test),grid.best_params_,grid]
#     index = index + 1

Fitting 14 folds for each of 12 candidates, totalling 168 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:  1.6min finished


GridSearchCV(cv=14, estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01],
                         'loss': ['hinge', 'log', 'modified_huber'],
                         'max_iter': [10000, 30000], 'n_jobs': [-1],
                         'tol': [0.0001]},
             verbose=1)

In [398]:
# model.fit(X_train, y_train)
# model.fit(X, y)

df_grid = pd.DataFrame(columns=['cv','score','best_params','grid'])
index = 0

# for k in range(5,15) :
for k in range(5,6) :
    grid = GridSearchCV(model, param_grid=params, cv=k, verbose = 1, n_jobs = -1)
    grid.fit(X_train,y_train)    
    df_grid.loc[index] = [k,grid.score(X_test,y_test),grid.best_params_,grid]
    index = index + 1

In [11]:
df_grid.sort_values(by = 'score', ascending=False, inplace=True)
df_grid.index = range(len(df_grid.index))

best_model = df_grid.loc[0,'grid']
df_grid.head(20)

NameError: name 'df_grid' is not defined

In [400]:
# model.best_params_
best_model.best_params_

{'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}

In [401]:
best_model.best_estimator_.score(X_test,y_test)
# model.best_estimator_.score(X_test,y_test)
# model.best_estimator_.score(X_train,y_train)
# model.best_estimator_.score(X,y)

0.7710843373493976

In [402]:
y_pred = best_model.predict(X_test)

In [421]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
import pandas as pd

# pd.DataFrame(
#     confusion_matrix(y_test, y_pred),
#     columns=['Positive', 'Negatif'],
#     index=['True', 'False']
# )

pd.DataFrame(
    confusion_matrix(y_test, y_pred)
    
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,6,1,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,2,8,0,0,0,0,0,0,0,0,0,0
6,0,0,1,1,0,0,5,1,0,0,0,0,0,0,0,0
7,0,0,0,2,1,0,1,12,1,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0
9,0,0,0,0,0,1,1,0,2,8,0,0,0,1,0,0


In [404]:
print("Classification report for - \n{}:\n{}\n".format(
    best_model.best_estimator_, metrics.classification_report(y_test, y_pred)))

In [406]:
images_a_predire = load_image_files("crop/")

In [407]:
predictions = best_model.predict(images_a_predire.data)

In [409]:
def load_crop_images_to_df(container_path):
    
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]
    
    images = []
    
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = skimage.io.imread(file)
            
            images.append(img)
            
    # images = np.array(images)

    return images

In [410]:
data = load_crop_images_to_df("crop/")

In [411]:
df = pd.DataFrame(data, columns=['images'])

In [412]:
df['type_de_dent'] = predictions

In [25]:
import joblib

# joblib.dump(best_model.best_estimator_, 'svm.joblib')
# joblib.dump(grid.best_estimator_, 'svm.joblib')

['svm.joblib']